Elección de las variables a usar en el modelo de recomendación


Para el modelo de recomndación se busca como resultado la recomendación en una lista de 5 juegos a partir de indicar el nombre de un juego o el id de un usuario, basado en un puntaje que se debe decidir cómo crearlo. Para ello, se revisan los dataframe df_reviews y df_items.

In [1]:
import pandas as pd
import numpy as np
import pyarrow as pa
import os
import pyarrow.parquet as pq

import warnings
warnings.filterwarnings("ignore")

In [2]:
 #Especificando la ruta del archivo Parquet
parquet_file_path = 'df_user_reviews.parquet'

# Leyendo el archivo Parquet en un DataFrame de Pandas
df_user_reviews = pq.read_table(parquet_file_path).to_pandas()

# Ahora, df_user_reviews contiene los datos del archivo Parquet
df_user_reviews.head(2)

,user_id,item_id,helpful,recommend,review,posted year,sentiment_analysis
0,76561197970982479,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2011,2
1,76561197970982479,22200,No ratings yet,True,It's unique and worth a playthrough.,2011,1


In [3]:
# Ruta del archivo Parquet
parquet_file_path = 'df_user_items_explode.parquet'

# Cargando el archivo Parquet en un DataFrame de Pandas
df_items = pq.read_table(parquet_file_path).to_pandas()

# Ahora, df contiene los datos del archivo Parquet
df_items.head(2)

,user_id,items_count,item_id,item_name,playtime_forever
0,76561197970982479,277,10,Counter-Strike,6.0
1,76561197970982479,277,20,Team Fortress Classic,0.0


In [4]:
def calcula_rating(fila):
    
    #Calcula una calificación basada en el análisis de sentimientos y la recomendación de review de juegos realizado por los usuarios.

    #Parámetros:
    #fila (dict): Un diccionario que contiene las siguientes claves:
       # - "sentiment_analysis" (int): La puntuación del análisis de sentimientos (0, 1 o 2).
        #- "reviews_recommend" (bool): Indica si las reseñas recomiendan.

    #Devuelve:
        #int o None: La calificación calculada como un número entero entre 1 y 5, o None si las entradas son inválidas
    if fila["sentiment_analysis"] == 0 and not fila["recommend"]:
        return 1
    elif fila["sentiment_analysis"] == 0 and fila["recommend"]:
        return 1
    elif fila["sentiment_analysis"] == 1 and not fila["recommend"]:
        return 2
    elif fila["sentiment_analysis"] == 1 and fila["recommend"]:
        return 3
    elif fila["sentiment_analysis"] == 2 and not fila["recommend"]:
        return 4
    elif fila["sentiment_analysis"] == 2 and fila["recommend"]:
        return 5
    else:
        return None

Con los datos disponible, se decide crear un puntaje (rating) para los juegos a partir de considerar el análisis de sentimiento de las reviews para cada juego y las recomendaciones del usuario para ese juego. Recordar que el análisis de sentimiento esta calificado como 0 (sentimiento negativo), 1 (sentimiento neutral) y 2 (sentimiento positivo) y las recomendaciones al juego son True (cuando lo recomienda) y False (cuando no recomienda el juego). Con esta información se busca hacer un puntaje del 1 al 5, siendo el 1 un juego no recomendado que cuentan con reviews con sentimientos negativos y 5 un juego recomendado y con reviews positivos. Se considera esta escala en función de la combinación de las dos variables de esta manera:

1 si el análisis de sentimiento es negativo ya sea que este recomendado o no (True o False)
2 si el análisis de sentimiento es neutral y no es recomendado (False)
3 si el análisis de sentimiento es neutral pero es recomendado (True)
4 si el análisis de sentimiento es positivo y no es recomendado (False)
5 si el análisis de sentimiento es positivo y es recomendado (True)

In [5]:
#Se aplica esta función en una nueva columna sobre el dataframe df_user_reviews

df_user_reviews['rating'] = df_user_reviews.apply(calcula_rating, axis=1)
df_user_reviews.head(10)

,user_id,item_id,helpful,recommend,review,posted year,sentiment_analysis,rating
0,76561197970982479,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2011,2,5
1,76561197970982479,22200,No ratings yet,True,It's unique and worth a playthrough.,2011,1,3
2,76561197970982479,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,2011,2,5
3,js41637,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2014,2,5
4,js41637,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,2013,2,5
5,js41637,239030,1 of 4 people (25%) found this review helpful,True,Very fun little game to play when your bored o...,2013,2,5
6,evcentric,370360,No ratings yet,True,"""Run for fun? What the hell kind of fun is that?""",2015,1,3
7,evcentric,237930,No ratings yet,True,"Elegant integration of gameplay, story, world ...",2014,2,5
8,evcentric,263360,No ratings yet,True,"Random drops and random quests, with stat poin...",2014,1,3
9,evcentric,107200,No ratings yet,True,Fun balance of tactics and strategy. Potentia...,2014,2,5


In [6]:
# creando un nuevo dataframe dejando solo las columnas 'user_id', 'reviews_item_id' y 'rating' 

df_1 = df_user_reviews[['user_id', 'item_id', 'rating']]
df_1.head(5)



,user_id,item_id,rating
0,76561197970982479,1250,5
1,76561197970982479,22200,3
2,76561197970982479,43110,5
3,js41637,251610,5
4,js41637,227300,5


In [7]:
# creando otro dataframe que contenga unicamente los 'item_id' y los nombres de los juegos

df_2 = df_items[['item_id', 'item_name']]

# borrando los dumplicados
df_2 = df_2.drop_duplicates()

# Cantidad de juegos
print(f'Hay un total de {len(df_2)} juegos')
df_2.head(2)



Hay un total de 10978 juegos


,item_id,item_name
0,10,Counter-Strike
1,20,Team Fortress Classic


In [8]:
# agregando los nombres de los juegos al primer dataframe:

df = df_1.merge(df_2, left_on="item_id", right_on="item_id", how='left')
df



,user_id,item_id,rating,item_name
0,76561197970982479,1250,5,Killing Floor
1,76561197970982479,22200,3,Zeno Clash
2,76561197970982479,43110,5,Metro 2033
3,js41637,251610,5,Barbie™ Dreamhouse Party™
4,js41637,227300,5,Euro Truck Simulator 2
...,...,...,...,...
48493,wayfeng,730,3,Counter-Strike: Global Offensive
48494,76561198251004808,253980,5,Enclave
48495,72947282842,730,3,Counter-Strike: Global Offensive
48496,ApxLGhost,730,5,Counter-Strike: Global Offensive


In [9]:
df.isnull().sum()


user_id         0
item_id         0
rating          0
item_name    5714
dtype: int64

In [10]:
# se revisan los reviews sin nombres de juegos en `df_items`
sin_juegos = df[df.isnull().any(axis=1)]
# Se calculan la cantidad de juegos en esta situación
print(f"Hay un total de {len(sin_juegos['item_id'].unique())} juegos con reviews pero que no estan en `df_reviews`")
sin_juegos

Hay un total de 256 juegos con reviews pero que no estan en `df_reviews`


,user_id,item_id,rating,item_name
27,76561198089393905,440,3,NaN
35,DJKamBer,570,5,NaN
39,DJKamBer,440,3,NaN
41,Rainbow-Dashie,440,5,NaN
46,devvonst,440,5,NaN
...,...,...,...,...
48451,BBiiiirr,440,5,NaN
48464,76561198209894493,440,3,NaN
48483,01shan,440,5,NaN
48487,76561198236893796,440,5,NaN


In [11]:
# borrando los datos nulos
df = df.dropna(subset=['item_id'])

# contando los registros que quedan
print(f'Quedan {df.shape[0]} registros')



Quedan 48498 registros


In [12]:
df.isnull().sum()


user_id         0
item_id         0
rating          0
item_name    5714
dtype: int64

In [13]:
df.dropna(inplace=True)
df

,user_id,item_id,rating,item_name
0,76561197970982479,1250,5,Killing Floor
1,76561197970982479,22200,3,Zeno Clash
2,76561197970982479,43110,5,Metro 2033
3,js41637,251610,5,Barbie™ Dreamhouse Party™
4,js41637,227300,5,Euro Truck Simulator 2
...,...,...,...,...
48493,wayfeng,730,3,Counter-Strike: Global Offensive
48494,76561198251004808,253980,5,Enclave
48495,72947282842,730,3,Counter-Strike: Global Offensive
48496,ApxLGhost,730,5,Counter-Strike: Global Offensive


In [14]:
df.isnull().sum()


user_id      0
item_id      0
rating       0
item_name    0
dtype: int64

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 42784 entries, 0 to 48497
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    42784 non-null  object
 1   item_id    42784 non-null  object
 2   rating     42784 non-null  int64 
 3   item_name  42784 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.6+ MB


In [16]:
# creando un último dataframe con las columnas necesarias para los modelos de recomendación.
df_3 = df[['user_id', 'item_name', 'rating']]
df_3.head(5)

,user_id,item_name,rating
0,76561197970982479,Killing Floor,5
1,76561197970982479,Zeno Clash,3
2,76561197970982479,Metro 2033,5
3,js41637,Barbie™ Dreamhouse Party™,5
4,js41637,Euro Truck Simulator 2,5


In [18]:
user_id_dtype = df_3['user_id'].dtype
user_id_dtype

dtype('O')

In [19]:
print("Longitud original del DataFrame:", len(df_3))
df_3 = df_3.dropna(subset=['user_id'])
print("Longitud del DataFrame después de eliminar filas con NaN en 'user_id':", len(df_3))


Longitud original del DataFrame: 42784
Longitud del DataFrame después de eliminar filas con NaN en 'user_id': 42784


In [20]:
df_3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 42784 entries, 0 to 48497
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    42784 non-null  object
 1   item_name  42784 non-null  object
 2   rating     42784 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.3+ MB


In [21]:
# Finalmente, se guarda el dataframe a utilizar en el modelo de recomnedación.

csv_file_path = 'df_3.csv'
df_3.to_csv(csv_file_path, index=False)
print(f"El DataFrame ha sido guardado exitosamente en: {csv_file_path}")


El DataFrame ha sido guardado exitosamente en: df_3.csv
